## Hotspot detection with unsupervised ML

## Detecting Hotspots in the month of May 2014 in NY

In [ ]:
! pip install geopy -q
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, accuracy_score, classification_report
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.datasets import load_digits
import datetime

In [ ]:
df = pd.read_csv("uber-raw-data-may14.csv")
df.head()

- Base : the TLC base company code affiliated with the Uber pickup REMOVE
- Date/Time: extract hour, day (year is not necessary, neither is the month)

In [ ]:
df.shape

In [ ]:
df['Date/Time'] = pd.to_datetime(df['Date/Time'],infer_datetime_format=True)
df['Day_of_Week'] = df['Date/Time'].dt.dayofweek
df['Hour'] = df['Date/Time'].dt.hour
df_clean=df.drop(['Date/Time','Base'], axis=1)

In [ ]:
sample = df.sample(1000)
sample=sample.sort_values("Hour", ascending=True)

fig = px.density_mapbox(sample, lat = 'Lat', lon = 'Lon', 
                        mapbox_style="carto-positron", zoom= 9, 
                        color_continuous_scale='viridis', opacity=0.6, radius=15)
fig.update_layout(
    width=1100,
    height=700)
fig.show()

In [ ]:
sample = df_clean.sample(250000)

## Calculating KMeans from the complete sample dataset

In [ ]:
scaler = StandardScaler()
X_all = scaler.fit_transform(sample)
X_all[:5]

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0)

kmeans.fit(X_all)

In [ ]:
cluster_centers_all = scaler.inverse_transform(kmeans.cluster_centers_)

In [ ]:
c=kmeans.predict(X_all)

In [ ]:
df_clean_all = sample.copy()
df_clean_all['cluster_id'] = c
fig_sample = df_clean_all.sample(2000)

In [ ]:
px.scatter_mapbox(fig_sample, lat = 'Lat', lon = 'Lon', mapbox_style="carto-positron", zoom= 10, color='cluster_id',width=1000,height=700)

**Creating clusters from the complete dataset doesn't appear to produce useful information about hotspots**

## Using DBScan to remove outliers before identifying pick-up hotspots

In [ ]:
X_all.shape

In [ ]:
# Instanciate DBSCAN and fit
db = DBSCAN(eps=0.20, min_samples=20)
db.fit(X_all)

In [ ]:
sample_db = sample.copy()
sample_db['Cluster_id'] =  db.labels_
sample_db['Cluster_id'].value_counts()/sample_db.shape[0]*100

We can aim at eliminating 10% of our data

In [ ]:
sample_db_f = sample_db.loc[sample_db['Cluster_id']>=0,['Lat','Lon','Day_of_Week','Hour']]
print(sample_db_f.shape[0])
sample_db_f.head()

In [ ]:
# check if the number of clusters is optimized
# calculate inertia and silhouette score for 12h

In [ ]:
# Let's create a loop that will collect the Within-sum-of-square (wcss) for each value K 
# Let's use .inertia_ parameter to get the within sum of square value for each value K 
wcss =  []
k = []
h=12
scaler_2 = StandardScaler()
sample_db_f = sample_db_f.sort_values('Hour')
X_h = sample_db_f.loc[sample_db_f['Hour']==h,['Lat','Lon']]
X_h_n = scaler_2.fit_transform(X_h)


for i in range (1,13): 
    kmeans = KMeans(n_clusters= i, random_state = 0)
    kmeans.fit(X_h)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))

In [ ]:
# Let's visualize using plotly
import plotly.express as px

# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show(renderer="iframe") # if using workspace

In [ ]:
# Import silhouette score
from sklearn.metrics import silhouette_score

# Computer mean silhouette score
sil = []
k = []


## Careful, you need to start at i=2 as silhouette score cannot accept less than 2 labels 
for i in range (2,13): 
    kmeans = KMeans(n_clusters= i, random_state = 0)
    kmeans.fit(X_h_n)
    sil.append(silhouette_score(X_h_n, kmeans.predict(X_h_n)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

In [ ]:
# Create a data frame 
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

# Create figure
fig = px.bar(data_frame=cluster_scores,  
             x=k, 
             y=cluster_scores.iloc[:, -1]
            )

# Add title and axis labels
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show(renderer="iframe") # if using workspace

In [ ]:
scaler_2 = StandardScaler()
labels_hour=[]
coord_hour={}
cluster_size={}
sample_db_f = sample_db_f.sort_values('Hour')
for i in range(0,24):
    # create sub data set to analyze by selecting by the hour and eliminating hour 
    X_i = sample_db_f.loc[sample_db_f['Hour']==i,['Lat','Lon']]
    # normalize
    X_i_n = scaler_2.fit_transform(X_i)
    # train model
    kmeans = KMeans(n_clusters=9, random_state=0)
    kmeans.fit(X_i_n)    
    # update dataframe with lat, lon, hour, and cluster#
    labels = kmeans.labels_.tolist()
    X_i['cluster_id'] = labels
    coordinates = kmeans.cluster_centers_
    coord_hour[i] = scaler_2.inverse_transform(coordinates)
    cluster_size[i]= X_i['cluster_id'].value_counts(sort=False).sort_index().to_numpy()
    labels_hour.extend(labels)

In [ ]:
clusters = np.array([[0, 0, 0, 0]])
for i in range(0,24):
    cluster_data = np.concatenate([np.full((9, 1), i), coord_hour[i], cluster_size[i].reshape(9,1)], axis=1)
    clusters = np.concatenate([clusters,cluster_data],axis=0)

clusters=np.delete(clusters, 0, 0)
np.set_printoptions(suppress=True)

In [ ]:
hour_hotspots = pd.DataFrame(clusters, columns = ['Hour','Lat', 'Lon','Size'])

In [ ]:
from geopy.geocoders import Nominatim
neighborhood=[]
for lat,lon in hour_hotspots.loc[:,['Lat','Lon']].to_numpy():
    geolocator = Nominatim(user_agent="NY_hotspots")
    location = geolocator.reverse(str(round(lat,3))+", "+str(round(lon,3)))
    try:
        neighborhood.append(location.raw['address']['neighbourhood'])
        continue
    except KeyError:
        pass
    try:
        neighborhood.append(location.raw['address']['quarter'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['aeroway'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['amenity'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['suburb'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['commercial'])
        continue
    except KeyError:
        pass
    neighborhood.append(str(lat)+" "+str(lon))

In [ ]:
hour_hotspots['Location'] = neighborhood

In [ ]:
fig = px.scatter_mapbox(hour_hotspots, lat = 'Lat', lon = 'Lon',size = 'Size',
                  color = 'Size', hover_name= 'Location',
                  hover_data={"Size": False,
                              "Hour":False,
                            "Lat": False,
                            "Lon": False
                        },
                  animation_frame='Hour',
                  size_max=50,
                  mapbox_style="carto-positron", zoom= 10, 
                  width=1000,height=700)

fig.write_html("Hour_map.html")
fig.show()

## Day-by-day pickup hotspots in NY using KMeans

In [ ]:
scaler_3 = StandardScaler()
labels_week=[]
coord_week={}
cluster_size_week={}
sample_db_f = sample_db_f.sort_values('Day_of_Week')
for i in range(0,7):
    # create sub data set to analyze by selecting by the hour and eliminating hour 
    X_i = sample_db_f.loc[sample_db_f['Day_of_Week']==i,['Lat','Lon']]
    # normalize
    X_i_n = scaler_3.fit_transform(X_i)
    # train model
    kmeans = KMeans(n_clusters=9, random_state=0)
    kmeans.fit(X_i_n)    
    # update dataframe with lat, lon, hour, and cluster#
    labels = kmeans.labels_.tolist()
    X_i['cluster_id'] = labels
    coordinates = kmeans.cluster_centers_
    coord_week[i] = scaler_3.inverse_transform(coordinates)
    cluster_size_week[i]= X_i['cluster_id'].value_counts(sort=False).sort_index().to_numpy()
    labels_week.extend(labels)

In [ ]:
clusters_week = np.array([[0, 0, 0, 0]])
for i in range(0,7):
    cluster_data = np.concatenate([np.full((9, 1), i), coord_week[i], cluster_size_week[i].reshape(9,1)], axis=1)
    clusters_week = np.concatenate([clusters_week,cluster_data],axis=0)

clusters_week=np.delete(clusters_week, 0,0)
np.set_printoptions(suppress=True)
clusters_week[:5]

In [ ]:
week_hotspots = pd.DataFrame(clusters_week, columns = ['Day_of_Week','Lat', 'Lon','Size'])

In [ ]:
neighborhood=[]
for lat,lon in week_hotspots.loc[:,['Lat','Lon']].to_numpy():
    geolocator = Nominatim(user_agent="NY_hotspots")
    location = geolocator.reverse(str(round(lat,3))+", "+str(round(lon,3)))
    try:
        neighborhood.append(location.raw['address']['neighbourhood'])
        continue
    except KeyError:
        pass
    try:
        neighborhood.append(location.raw['address']['quarter'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['aeroway'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['amenity'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['suburb'])
        continue
    except KeyError:
        pass
    try: 
        neighborhood.append(location.raw['address']['commercial'])
        continue
    except KeyError:
        pass
    neighborhood.append(str(lat)+" "+str(lon))

In [ ]:
week_hotspots['Location'] = neighborhood

In [ ]:
px.scatter_mapbox(week_hotspots, lat = 'Lat', lon = 'Lon',size = 'Size',
                  color = 'Size',
                  animation_frame='Day_of_Week',
                  size_max=50,
                  mapbox_style="carto-positron", zoom= 10, 
                  width=1000,height=700)